In [1]:
import json
import sys
from urllib.parse import urlparse, parse_qs, urljoin

import requests as rq
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import os
import re

dir_path = "C:\\Users\\user\\Desktop\\python\\datamining\\컷툰_작가_시간\\" ## 수정부분


In [2]:
NAVER_URL = "https://comic.naver.com/index.nhn"
TOP_URL = "https://comic.naver.com/webtoon/weekday.nhn"

In [3]:
def save(data, file_name):
    file = open(file_name, 'a')
    file.write(data + '\n')

In [4]:
def get_daylywebtoons():
    '''
    요일 웹툰을 수집
    '''

    webtoon_main_url = TOP_URL

    res = rq.get(webtoon_main_url)

    main_soup = BeautifulSoup(res.content, 'lxml')

    webtoon_links = [{"title": a_tag.get('title'), "link": urljoin(NAVER_URL, a_tag.get('href'))}
                      for a_tag in main_soup.select('.daily_all a.title')]

    return webtoon_links


In [5]:
def make_link(webtoon_url, page_count):
    return webtoon_url + '&page=' + str(page_count)


def get_qs(url, key):
    url_query = urlparse(url).query
    query_dict = parse_qs(url_query)
    value = query_dict[key][0]
    return value

In [6]:

def get_all_webtoon(webtoon, is_save):
    '''
    해당 웹 툰의  1화 ~ 마지막까지 수집
    '''
    page_count = 1

    target_webtoons = list()
    webtoon_url = webtoon['link']
    webtoon_title = webtoon['title']

    webtoon_id = get_qs(webtoon_url, 'titleId')
    weekday = get_qs(webtoon_url, 'weekday')

    is_unlast = True

    while is_unlast:
        link = make_link(webtoon_url, page_count)

        target_webtoon_res = rq.get(link)
        webtoon_soup = BeautifulSoup(target_webtoon_res.content, 'lxml')
        a_tags = webtoon_soup.select('.viewList td.title a')

        for a_tag in a_tags:
            t = a_tag.text.replace('\n', '').replace('\r', '').replace('\t', '')
            h = urljoin(NAVER_URL, a_tag.get('href'))

            if h not in target_webtoons:
                target_webtoons.append(h)
            else:
                is_unlast = False

        page_count += 1

    if is_save:
        for webtoon in target_webtoons:
            save(webtoon_title + ':' + webtoon, 'all_webtoons.txt')

    return webtoon_title.strip(), webtoon_id, weekday, target_webtoons



In [7]:
def data_parse2(soup, url):
    title = soup.title.text.split(':')[0].lstrip(" ").rstrip(" ")
    no_title = soup.select('.tit_area .view h3')[0].text
    no_title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』姊\\‘|\(\)\[\]\<\>`\'… 【】▲▶◇ⓒⓒ■~★○●◆♥》└ ┘]', '', no_title)
    no_title = re.sub('下', '하', no_title)
    no_title = re.sub('上', '상', no_title)
    no_title = re.sub("\u200b","",no_title)
    no_title = re.sub("\u200b","",no_title)
    
    no_date = soup.find("dd",{"class":"date"}).text
    no_time = soup.find("input",{"name":"itemDt"}).get("value").split(" ")[1]
  
    num_cut = soup.find("div",{"class":"wt_viewer"}).find_all("img",{"alt":"comic content"})
    writer_info = soup.find("div",{"class":"writer_info"}).find("p").text
    
    check=0
    if len(writer_info) != 1 : 
        check +=1
        
    cut_writer = [title,no_title,no_date,no_time,len(num_cut),check]
 
    return(cut_writer)

In [10]:
#나이트런
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
      #  os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            print(webtoon_page_soup)
            part = data_parse2(webtoon_page_soup, webtoon_page)
            print(part)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+ webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>신의 탑 :: 네이버 만화</title>
<meta content="신의 탑 - 3부 12화" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/webtoon/183559/431/thumbnail_600x315_51a95c24-4110-4546-828e-ef84834abfa3.jpg" property="og:image"/>
<meta content="3부 12화" property="og:description"/>
<meta content="https://comic.naver.com/webtoon/detail.nhn?titleId=183559&amp;no=431&amp;weekday=mon" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Co

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>신의 탑 :: 네이버 만화</title>
<meta content="신의 탑 - 3부 11화" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/webtoon/183559/430/thumbnail_600x315_78461f0d-25f9-4179-90b3-84d74bacbc39.jpg" property="og:image"/>
<meta content="3부 11화" property="og:description"/>
<meta content="https://comic.naver.com/webtoon/detail.nhn?titleId=183559&amp;no=430&amp;weekday=mon" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Co

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>신의 탑 :: 네이버 만화</title>
<meta content="신의 탑 - 3부 10화" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/webtoon/183559/429/optimized_thumbnail_600x315_43fb3b20-211f-4711-b22f-f1eaf4cbf745.jpg" property="og:image"/>
<meta content="3부 10화" property="og:description"/>
<meta content="https://comic.naver.com/webtoon/detail.nhn?titleId=183559&amp;no=429&amp;weekday=mon" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>신의 탑 :: 네이버 만화</title>
<meta content="신의 탑 - 3부 9화" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/webtoon/183559/428/optimized_thumbnail_600x315_490aa556-460a-4a2d-becc-092fae38ef06.jpg" property="og:image"/>
<meta content="3부 9화" property="og:description"/>
<meta content="https://comic.naver.com/webtoon/detail.nhn?titleId=183559&amp;no=428&amp;weekday=mon" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-e

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>신의 탑 :: 네이버 만화</title>
<meta content="신의 탑 - 3부 8화" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/webtoon/183559/427/optimized_thumbnail_600x315_79d6d555-ce67-43e1-aad3-6d063cc0b3b0.jpg" property="og:image"/>
<meta content="3부 8화" property="og:description"/>
<meta content="https://comic.naver.com/webtoon/detail.nhn?titleId=183559&amp;no=427&amp;weekday=mon" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-e

...

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>신의 탑 :: 네이버 만화</title>
<meta content="신의 탑 - 3부 5화" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/webtoon/183559/424/optimized_thumbnail_600x315_bcbec43e-8ba5-499b-b77e-c6b92ac17cce.jpg" property="og:image"/>
<meta content="3부 5화" property="og:description"/>
<meta content="https://comic.naver.com/webtoon/detail.nhn?titleId=183559&amp;no=424&amp;weekday=mon" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-e

['신의 탑', '3부4화', '2019.03.31', '23:09:00', 63, 1]
<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>신의 탑 :: 네이버 만화</title>
<meta content="신의 탑 - 3부 3화" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/webtoon/183559/422/optimized_thumbnail_600x315_26bcb0a4-7b0f-4f52-9f95-fa0da79d5852.jpg" property="og:image"/>
<meta content="3부 3화" property="og:description"/>
<meta content="https://comic.naver.com/webtoon/detail.nhn?titleId=183559&amp;no=422&amp;weekday=mon" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<meta content="text/javascript" http-equiv="Cont

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>신의 탑 :: 네이버 만화</title>
<meta content="신의 탑 - 3부 2화" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/webtoon/183559/421/optimized_thumbnail_600x315_4535f7f1-a9bc-48d4-a67d-fa633e4f8221.jpg" property="og:image"/>
<meta content="3부 2화" property="og:description"/>
<meta content="https://comic.naver.com/webtoon/detail.nhn?titleId=183559&amp;no=421&amp;weekday=mon" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-e

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>신의 탑 :: 네이버 만화</title>
<meta content="신의 탑 - 3부 1화" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/webtoon/183559/420/optimized_thumbnail_600x315_6cf3ea8d-ad05-4348-8521-5604aabbf3a5.jpg" property="og:image"/>
<meta content="3부 1화" property="og:description"/>
<meta content="https://comic.naver.com/webtoon/detail.nhn?titleId=183559&amp;no=420&amp;weekday=mon" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-e

KeyboardInterrupt: 

In [15]:
#미시령
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[96:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+ webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

IndexError: list index out of range

In [15]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[61:66]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [18]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[78:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'C:\\Users\\user\\Desktop\\python\\datamining\\컷툰_작가_시간\\신암행어사'

In [19]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[83:84]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [20]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[85:87]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [21]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[88:99]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [8]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[100:119]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [11]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[119:120]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [14]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[133:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'C:\\Users\\user\\Desktop\\python\\datamining\\컷툰_작가_시간\\덴마'

In [15]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[154:156]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [16]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[157:158]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [22]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[166:167]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [20]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[168:169]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [23]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[170:177]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [24]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[178:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

OSError: [WinError 123] 파일 이름, 디렉터리 이름 또는 볼륨 레이블 구문이 잘못되었습니다: 'C:\\Users\\user\\Desktop\\python\\datamining\\컷툰_작가_시간\\여기가 씨름부입니까?'

In [25]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[180:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

IndexError: list index out of range

In [27]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[186:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'C:\\Users\\user\\Desktop\\python\\datamining\\컷툰_작가_시간\\유미의 세포들'

In [28]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[188:193]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [29]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[195:197]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [30]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[198:201]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [31]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[202:203]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [35]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[211:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'C:\\Users\\user\\Desktop\\python\\datamining\\컷툰_작가_시간\\모죠의 일지'

In [36]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[219:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'C:\\Users\\user\\Desktop\\python\\datamining\\컷툰_작가_시간\\덴마'

In [38]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[226:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'C:\\Users\\user\\Desktop\\python\\datamining\\컷툰_작가_시간\\신암행어사'

In [39]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[228:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'C:\\Users\\user\\Desktop\\python\\datamining\\컷툰_작가_시간\\내 어린고양이와 늙은개 (재)'

In [41]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[245:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'C:\\Users\\user\\Desktop\\python\\datamining\\컷툰_작가_시간\\Here U Are'

In [42]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[249:]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")

In [43]:
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    for webtoon in webtoons[248:249]:
        webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoon, False)## 수정부분
        os.mkdir(dir_path + webtoon_title)
        total =[]
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            part = data_parse2(webtoon_page_soup, webtoon_page)
            total.append(part)
        txttotal = pd.DataFrame(total)
        txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949")